In [21]:
import pdal
import sys
import os
from collections import defaultdict
# Get the number of logical CPUs in the system
processors = os.cpu_count() - 1
import json


sys.path.append("../PythonScripts")
from glob import glob
import open3d as o3d
import os
import numpy as np
import pandas as pd
import multiprocessing as mp
from pipeline_functions import downscale_las, get_scales
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.spatial import KDTree
sns.set()

In [2]:
ROOT = """/home/sspiegel/CapstoneData/WashingtonDC/OpenDataDC_LAS_Point_Cloud_2020_Block1/Block1/2117.las"""

## Load in base point cloud

In [3]:
pipeline_json = f"""
[
    {{
        "type": "readers.las",
        "filename": "{ROOT}"
    }}
]
"""

pipeline = pdal.Pipeline(pipeline_json)

# Execute the pipeline
# This will process the data according to the stages defined in the pipeline
pipeline.execute()

xy = pipeline.arrays[0]



## Get scaling factors

### Downscale point clouds: Methodology

* Downscale point clouds based on multiscaling factor
* The scaling factor chosen from [Thomas Hugues et. al](https://ieeexplore.ieee.org/document/8490990/)
* Parameters
  * initial radius ($r_0$): 0.1
  * Number of scales ($S$): 8
  * Base exponent for expanding radius ($\gamma$): 2
  * scaling factor of grid ($\rho$): 5
  * radius of sphere for scale $s$: $r_s = r_0 * \gamma^{s}$
  * downsampled voxel size: $\frac{r_s}{\rho}$

* Use multiprocessing to speed up process

In [4]:
scls = get_scales() # Use base paprameters from function

In [5]:
scls_grid = [(f"""{ROOT}""",g[1]) for g in scls] # Build arguments for starmap function in multiprocessing

In [7]:
%%timeit
with mp.Pool(processes=processors) as pool:
    results = pool.starmap(downscale_las, scls_grid)

8.17 s ± 1 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Grab downscaled files

In [8]:
glbs = glob("""/home/sspiegel/CapstoneData/WashingtonDC/OpenDataDC_LAS_Point_Cloud_2020_Block1/Block1/*_downsampled_*.las""")

In [17]:
glbs

['/home/sspiegel/CapstoneData/WashingtonDC/OpenDataDC_LAS_Point_Cloud_2020_Block1/Block1/2117_downsampled_0_02.las',
 '/home/sspiegel/CapstoneData/WashingtonDC/OpenDataDC_LAS_Point_Cloud_2020_Block1/Block1/2117_downsampled_2_56.las',
 '/home/sspiegel/CapstoneData/WashingtonDC/OpenDataDC_LAS_Point_Cloud_2020_Block1/Block1/2117_downsampled_0_04.las',
 '/home/sspiegel/CapstoneData/WashingtonDC/OpenDataDC_LAS_Point_Cloud_2020_Block1/Block1/2117_downsampled_0_64.las',
 '/home/sspiegel/CapstoneData/WashingtonDC/OpenDataDC_LAS_Point_Cloud_2020_Block1/Block1/2117_downsampled_1_28.las',
 '/home/sspiegel/CapstoneData/WashingtonDC/OpenDataDC_LAS_Point_Cloud_2020_Block1/Block1/2117_downsampled_0_08.las',
 '/home/sspiegel/CapstoneData/WashingtonDC/OpenDataDC_LAS_Point_Cloud_2020_Block1/Block1/2117_downsampled_0_32.las',
 '/home/sspiegel/CapstoneData/WashingtonDC/OpenDataDC_LAS_Point_Cloud_2020_Block1/Block1/2117_downsampled_0_16.las']

## Load in Downsampled point clouds to calculate features

In [18]:
pc_dict = []
for s in scls:
    pc_dict.append({
      "r" : s[0],
     "grid_size" : s[1],
     "file_name" : f"""/home/sspiegel/CapstoneData/WashingtonDC/OpenDataDC_LAS_Point_Cloud_2020_Block1/Block1/2117_downsampled_{str(s[1]).split('.')[0]}_{str(s[1]).split('.')[1]}.las"""
    })
    


In [19]:
pc_dict

[{'r': 0.1,
  'grid_size': 0.02,
  'file_name': '/home/sspiegel/CapstoneData/WashingtonDC/OpenDataDC_LAS_Point_Cloud_2020_Block1/Block1/2117_downsampled_0_02.las'},
 {'r': 0.2,
  'grid_size': 0.04,
  'file_name': '/home/sspiegel/CapstoneData/WashingtonDC/OpenDataDC_LAS_Point_Cloud_2020_Block1/Block1/2117_downsampled_0_04.las'},
 {'r': 0.4,
  'grid_size': 0.08,
  'file_name': '/home/sspiegel/CapstoneData/WashingtonDC/OpenDataDC_LAS_Point_Cloud_2020_Block1/Block1/2117_downsampled_0_08.las'},
 {'r': 0.8,
  'grid_size': 0.16,
  'file_name': '/home/sspiegel/CapstoneData/WashingtonDC/OpenDataDC_LAS_Point_Cloud_2020_Block1/Block1/2117_downsampled_0_16.las'},
 {'r': 1.6,
  'grid_size': 0.32,
  'file_name': '/home/sspiegel/CapstoneData/WashingtonDC/OpenDataDC_LAS_Point_Cloud_2020_Block1/Block1/2117_downsampled_0_32.las'},
 {'r': 3.2,
  'grid_size': 0.64,
  'file_name': '/home/sspiegel/CapstoneData/WashingtonDC/OpenDataDC_LAS_Point_Cloud_2020_Block1/Block1/2117_downsampled_0_64.las'},
 {'r': 6.4

In [25]:
test = pc_dict[0]

pipeline_json = {"pipeline" :
[
    {
        "type": "readers.las",
        "filename": test['file_name']
    }
]
}

pipeline = pdal.Pipeline(json.dumps(pipeline_json))

# Execute the pipeline
# This will process the data according to the stages defined in the pipeline
pipeline.execute()

xy_downscale = pipeline.arrays[0]
xy_downscale = np.vstack((xy_downscale['X'], xy_downscale['Y'], xy_downscale['Z'])).T

In [26]:

# xy_downscale

array([[3.9740307e+05, 1.3700000e+05, 5.6220000e+01],
       [3.9743951e+05, 1.3700000e+05, 1.7510000e+01],
       [3.9744206e+05, 1.3700000e+05, 1.5920000e+01],
       ...,
       [3.9819001e+05, 1.3779998e+05, 2.6810000e+01],
       [3.9819723e+05, 1.3779998e+05, 4.0380000e+01],
       [3.9819777e+05, 1.3779998e+05, 4.0640000e+01]], shape=(4856909, 3))

In [39]:
pipeline_json = f"""
[
    {{
        "type": "readers.las",
        "filename": "{ROOT}"
    }}
]
"""

pipeline = pdal.Pipeline(pipeline_json)

# Execute the pipeline
# This will process the data according to the stages defined in the pipeline
pipeline.execute()

# Access the point data as a NumPy array
# The 'arrays' attribute of the pipeline holds the output data


4871870